# Imports

In [1]:
print("HELLO")

HELLO


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf 
import os
import mediapipe as mp

2024-09-27 15:04:46.561434: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-27 15:04:46.567680: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-27 15:04:46.580524: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-27 15:04:46.606270: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-27 15:04:46.614306: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-27 15:04:46.633634: I tensorflow/core/platform/cpu_feature_gu

In [3]:
mp_holistic = mp.solutions.holistic 
mp_drawing = mp.solutions.drawing_utils

In [15]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(245, 117, 76), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(245, 44, 250), thickness=1, circle_radius=1)
                             ) 

In [5]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Color conversion BGR to RGB
    image.flags.writeable = False  # Image is no longer writeable
    results = model.process(image)  # Make prediction
    image.flags.writeable = True  # Image writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # Color conversion RGB to BGR
    return image, results

In [6]:
def extract_keypoints(results):
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() \
                if results.face_landmarks else np.zeros(468*3)
    return np.concatenate([face])

In [16]:
cam = cv2.VideoCapture(0)

# Access mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cam.isOpened():
    
        # Read feed
        ret, frame = cam.read()
        
        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        # print(result)

        # Draw Landmarks
        draw_landmarks(image, results)
        
        # Show to screen
        cv2.imshow("OpenCV Feed", image)
        
        # Break gracefully
        if cv2.waitKey(10)& 0xFF == ord('q'):
            break

    # Release the webcam and close all OpenCV windows
    cam.release()
    cv2.destroyAllWindows()

I0000 00:00:1727429345.602614   15468 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1727429345.603942   15969 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: Mesa Intel(R) HD Graphics 630 (KBL GT2)
W0000 00:00:1727429345.685373   15960 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1727429345.722024   15959 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1727429345.723498   15962 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1727429345.723505   15961 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback t

In [8]:
# plt.imshow(frame)

In [9]:
# draw_landmarks(frame, results)
# plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

In [10]:
# results.face_landmarks.landmark

# Folder setup for data storing

In [11]:
# Path for exported data
DATA_PATH = os.path.join("FACE_DATA")

# Actions to detect
actions = np.array(["happy", "sad", 'angry'])

# Thirty videos worth of data
no_sequences = 30

# Frames 
sequence_length = 30

In [12]:
# # Make directory for each video and frames
# for action in actions:
#     for sequence in range(no_sequences):
#         try:
#             os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
#         except:
#             pass

In [13]:
# cam = cv2.VideoCapture(0)

# if not cam.isOpened():
#     print("Error: Could not open webcam.")
#     exit()
    
# # Access mediapipe model
# with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#     # while cam.isOpened():

#     # Loop through actions
#     for action in actions:
#         # Loop through video/ sequences
#         for sequence in range(no_sequences):
#             # Loop through frames
#             for frame_num in range(sequence_length):
                
#                 # Read feed
#                 ret, frame = cam.read()
                
#                 # Make detections
#                 image, results = mediapipe_detection(frame, holistic)
#                 # print(result)
        
#                 # Draw Landmarks
#                 draw_landmarks(image, results)

#                 # Apply collection logic
#                 if frame_num == 0:
#                     cv2.putText(image, "Starting collection", (120, 200),
#                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1, cv2.LINE_AA)
#                     cv2.putText(image, "Collecting frames for {} Video Number {}".format(action, sequence), (15, 12),
#                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
#                     cv2.waitKey(2000)
#                 else:
#                     cv2.putText(image, "Collecting frames for {} Video Number {}".format(action, sequence), (15, 12),
#                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
#                 # New Export key points    
#                 keypoints = extract_keypoints(results)
#                 npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
#                 np.save(npy_path, keypoints)
                
#                 # Show to screen
#                 cv2.imshow("OpenCV Feed", image)
            
#                 # Break gracefully
#                 if cv2.waitKey(10)& 0xFF == ord('q'):
#                     break

#     # Release the webcam and close all OpenCV windows
#     cam.release()
#     cv2.destroyAllWindows()

I0000 00:00:1727428789.545801   15468 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1727428789.563887   15570 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 23.2.1-1ubuntu3.1~22.04.2), renderer: Mesa Intel(R) HD Graphics 630 (KBL GT2)
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1727428789.678795   15554 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1727428789.724015   15558 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1727428789.730026   15553 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1727428789.731164   15556 inference_feedback_manager.cc:114] Feedback manager requires a model with a sing